In [1]:
import numpy as np
from datetime import datetime

def classify_expenses(expenses):
    categorized_expenses = {"Needs": [], "Wants": [], "Savings": []}
    
    needs_categories = {"health", "education", "rent", "groceries"}
    wants_categories = {"shopping", "misc"}
    
    for item, amount, category in expenses:
        if category in needs_categories:
            categorized_expenses["Needs"].append((item, amount))
        elif category in wants_categories:
            categorized_expenses["Wants"].append((item, amount))
        else:
            categorized_expenses["Savings"].append((item, amount))
    
    return categorized_expenses

def calculate_amortized_payment(target_amount, interest_rate, months):
    if interest_rate == 0:
        return target_amount / months
    r = interest_rate / 100 / 12  # Convert annual interest rate to monthly
    return (target_amount * r) / (1 - (1 + r) ** -months)

def optimize_spending(income, expenses, target_amount, interest_rate, start_date, end_date):
    categorized_expenses = classify_expenses(expenses)
    
    total_needs = sum(amount for _, amount in categorized_expenses["Needs"])
    total_wants = sum(amount for _, amount in categorized_expenses["Wants"])
    total_savings = sum(amount for _, amount in categorized_expenses["Savings"])
    
    start = datetime.strptime(start_date, "%Y-%m-%d")
    end = datetime.strptime(end_date, "%Y-%m-%d")
    months = max(1, (end.year - start.year) * 12 + (end.month - start.month))
    
    monthly_savings_needed = calculate_amortized_payment(target_amount, interest_rate, months)
    discretionary_income = income - total_needs
    optimized_wants = max(0, discretionary_income - monthly_savings_needed)
    optimized_savings = discretionary_income - optimized_wants
    
    recommendations = {
        "Optimized Needs": total_needs,
        "Optimized Wants": optimized_wants,
        "Optimized Savings": optimized_savings,
        "Monthly Savings Target": monthly_savings_needed,
        "Suggested Adjustments": {},
        "Next Month Limits": {}
    }
    
    if optimized_savings < monthly_savings_needed:
        excess_needed = monthly_savings_needed - optimized_savings
        cut_wants = min(total_wants, excess_needed)
        recommendations["Suggested Adjustments"]["Reduce Wants"] = cut_wants
        optimized_wants -= cut_wants
        optimized_savings += cut_wants
        
        # Additional insights for adjustments
        if optimized_savings < monthly_savings_needed:
            recommendations["Suggested Adjustments"]["Increase Income"] = f"Consider increasing income by at least {monthly_savings_needed - optimized_savings:.2f} to meet savings goal."
            recommendations["Suggested Adjustments"]["Reduce Needs"] = "Evaluate fixed expenses to identify possible reductions, such as switching to a cheaper plan or optimizing grocery shopping."
    else:
        recommendations["Suggested Adjustments"]["Success"] = "Good going! You are on the right track!"
    
    # Recalculate new amortized payments for the next month
    new_target_amount = target_amount - optimized_savings
    new_months = months - 1 if months > 1 else 1
    new_monthly_savings_needed = calculate_amortized_payment(new_target_amount, interest_rate, new_months)
    
    # Ensure next month limits sum up to income
    total_next_month = total_needs + optimized_wants + new_monthly_savings_needed
    scale_factor = income / total_next_month if total_next_month > 0 else 1
    
    recommendations["Next Month Limits"] = {
        "Needs": total_needs * scale_factor,
        "Wants": optimized_wants * scale_factor,
        "Savings": new_monthly_savings_needed * scale_factor
    }
    
    return recommendations

# Example Usage
income = 5000
expenses = [
    ("Rent", 1500, "rent"),
    ("Groceries", 500, "groceries"),
    ("Dining Out", 300, "misc"),
    ("Shopping", 400, "shopping"),
    ("Health Insurance", 600, "health"),
    ("Education Loan", 700, "education")
]
target_amount = 10000
interest_rate = 5  # Annual interest rate in percentage
start_date = "2024-01-01"
end_date = "2024-12-31"

result = optimize_spending(income, expenses, target_amount, interest_rate, start_date, end_date)
print(result)

{'Optimized Needs': 3300, 'Optimized Wants': 768.0243233339511, 'Optimized Savings': 931.9756766660489, 'Monthly Savings Target': 931.9756766660489, 'Suggested Adjustments': {'Success': 'Good going! You are on the right track!'}, 'Next Month Limits': {'Needs': 3302.815814505495, 'Wants': 768.6796609188652, 'Savings': 928.5045245756396}}
